In [1]:
import glob
import os
import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz

In [2]:
path = "/home/jorgher/Documents/faceRec/CFEE/"
files = sorted(glob.glob(path+'*.csv'))
files = files[:7]
emotions = ["neutral",'happy','sad','fearful','angry','surprised','disgusted']
dbs = []
for i,f in enumerate(files):
    db = pd.read_csv(f)
    db = db[['D1','D2','D3','D4','D5','D6','D7','A0','A1','A2','A3','A4','A5']]
    db['Emotions'] = emotions[i]
    dbs.append(db)
    
db_Train = (pd.concat(dbs[i][:130] for i in range(len(files))))
db_Test = (pd.concat(dbs[i][130:230] for i in range(len(files))))
db = (pd.concat(dbs[i][:] for i in range(len(files))))
X = db_Train[['D1','D2','D3','D4','D5','D6','D7','A0','A1','A2','A3','A4','A5']]
Y = db_Train['Emotions']
#X_Norm =(X-X.mean())/X.std()
#X_Cov = np.cov(X_Norm)
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, Y)
score = clf.score(X,Y)
print(score)

1.0


In [3]:
# Randomly sample 70% of your dataframe
db_0_7 = db_Test.sample(frac=0.7)
# Randomly sample 7 elements from your dataframe
db_7 = db_Test.sample(n=7)
#For either approach above, you can get the rest of the rows by doing:
db_rest = db.loc[~db.index.isin(db_0_7.index)]

In [4]:
db_7

,D1,D2,D3,D4,D5,D6,D7,A0,A1,A2,A3,A4,A5,Emotions
228,22.022716,25.019992,27.856777,6.000000,7.280110,21.023796,63.031738,1.128952,0.305226,1.169568,0.247447,0.634492,1.139670,disgusted
219,22.090722,26.019224,27.856777,7.071068,8.062258,19.000000,65.069194,0.826071,0.360164,1.035011,0.270410,0.513799,0.949251,sad
196,29.068884,25.079872,29.154759,8.062258,8.246211,26.076810,81.221918,0.885398,0.279903,1.095133,0.350306,0.551013,0.900380,happy
132,11.045361,25.495098,24.351591,8.062258,6.000000,17.117243,56.080300,0.983298,0.381704,1.056733,0.161902,0.594586,0.983298,sad
225,33.060551,16.763055,20.248457,10.000000,10.198039,28.071338,49.091751,1.496608,0.220044,1.524910,0.528434,1.050200,1.496608,surprised
135,35.014283,28.000000,31.780497,13.038405,13.038405,27.018512,53.037722,1.379794,0.262076,0.931754,0.399176,1.007594,1.379794,fearful
195,24.083189,20.000000,22.360680,7.071068,6.082763,22.022716,70.178344,0.919720,0.338075,1.224257,0.312957,0.600615,0.971987,disgusted


In [5]:
test = db_7[['D1','D2','D3','D4','D5','D6','D7','A0','A1','A2','A3','A4','A5']]
for i in range(len(test)):
    #print(test.iloc[[i]])
    print(clf.predict(test.iloc[[i]]))
    #print(clf.predict_proba(test.iloc[[i]]))
    
   

['angry']
['sad']
['happy']
['disgusted']
['fearful']
['fearful']
['disgusted']


## Generando el arbol

In [6]:
def encode_target(df, target_column):
    """Add column to df with integers for the target.

    Args
    ----
    df -- pandas DataFrame.
    target_column -- column to map to int, producing
                     new Target column.

    Returns
    -------
    df_mod -- modified DataFrame.
    targets -- list of target names.
    """
    df_mod = df.copy()
    targets = df_mod[target_column].unique()
    map_to_int = {name: n for n, name in enumerate(targets)}
    df_mod["Target"] = df_mod[target_column].replace(map_to_int)

    return (df_mod, targets)
    

In [7]:
df2, targets = encode_target(db, "Emotions")
print("* df2.head()", df2[["Target", "Emotions"]].head(),
      sep="\n", end="\n\n")
print("* df2.tail()", df2[["Target", "Emotions"]].tail(),
      sep="\n", end="\n\n")
print("* targets", targets, sep="\n", end="\n\n")


* df2.head()
   Target Emotions
0       0  neutral
1       0  neutral
2       0  neutral
3       0  neutral
4       0  neutral

* df2.tail()
     Target   Emotions
225       6  disgusted
226       6  disgusted
227       6  disgusted
228       6  disgusted
229       6  disgusted

* targets
['neutral' 'happy' 'sad' 'fearful' 'angry' 'surprised' 'disgusted']



In [8]:
features = list(df2.columns[:13])
print("* features:", features, sep="\n")
Etype={'neutral': 0, 'happy': 1,
        'sad': 2,'fearful': 3,'angry': 4,
        'surprised': 5,'disgusted': 6}
ERtype={0:'neutral', 1: 'happy',
        2: 'sad',3: 'fearful',4:'angry',
        5: 'surprised',6: 'disgusted'}

* features:
['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'A0', 'A1', 'A2', 'A3', 'A4', 'A5']


In [9]:
e = ERtype[0]
print(e)

neutral


In [10]:
y = df2["Target"]
X = df2[features]
dt = DecisionTreeClassifier(min_samples_split=20, random_state=99)
dt.fit(X, y)
print(db_7['Emotions'])
test = db_7[['D1','D2','D3','D4','D5','D6','D7','A0','A1','A2','A3','A4','A5']]
for i in range(len(test)):
    #print(test.iloc[[i]])
    print(dt.predict(test.iloc[[i]]))
    #print(clf.predict_proba(test.iloc[[i]]))

228    disgusted
219          sad
196        happy
132          sad
225    surprised
135      fearful
195    disgusted
Name: Emotions, dtype: object
[6]
[2]
[1]
[3]
[5]
[3]
[6]


In [11]:
def visualize_tree(tree, feature_names):
    """Create tree png using graphviz.

    Args
    ----
    tree -- scikit-learn DecsisionTree.
    feature_names -- list of feature names.
    """
    with open("dt.dotA", 'w') as f:
        export_graphviz(tree, out_file=f,
                        feature_names=feature_names)

    command = ["dot", "-Tpng", "dt.dot", "-o", "dt.png"]
    try:
        subprocess.check_call(command)
    except:
        exit("Could not run dot, ie graphviz, to "
             "produce visualization")

In [12]:
visualize_tree(dt, features)
# Aparentemente la grafica no se ve en jupiter pero genera los archivos .dot

## Probando con diferentes clasificadores y verificando su score

In [13]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

names =  ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process"]
names1 = ["Decision Tree", "Random Forest", "Neural Net", "AdaBoost"]
names2 = ["Naive Bayes", "QDA"]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0))]
classifiers1 = [DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=5),
    MLPClassifier(alpha=1),
    AdaBoostClassifier()]
classifiers2 = [GaussianNB(),
    QuadraticDiscriminantAnalysis()]

In [14]:
for name, clf in zip(names2, classifiers2):
        print(name)
        clf.fit(X, y)
        score = clf.score(X, y)
        print(score)
        for i in range(len(test)):
            #print(test.iloc[[i]])
            print(clf.predict(test.iloc[[i]]))

Naive Bayes
0.624223602484
[6]
[2]
[1]
[2]
[5]
[5]
[6]
QDA
0.726086956522
[4]
[2]
[1]
[2]
[6]
[5]
[6]


In [15]:
for name, clf in zip(names1, classifiers1):
        print(name)
        clf.fit(X, y)
        score = clf.score(X, y)
        print(score)
        for i in range(len(test)):
            #print(test.iloc[[i]])
            print(clf.predict(test.iloc[[i]]))

Decision Tree
0.601242236025
[4]
[2]
[1]
[2]
[5]
[3]
[4]
Random Forest
0.664596273292
[6]
[4]
[1]
[2]
[5]
[5]
[4]
Neural Net
0.56149068323
[6]
[2]
[6]
[2]
[5]
[5]
[1]
AdaBoost
0.567080745342
[6]
[1]
[1]
[2]
[5]
[5]
[1]


In [22]:
for name, clf in zip(names, classifiers):
        print(name)
        clf.fit(X, y)
        score = clf.score(X, y)
        print(score)
        con = 0
        for i in range(len(test)):
            e = clf.predict(test.iloc[[i]])
            print(ERtype[e[0]])
            if ERtype[e[0]] == db_7['Emotions']:
                con = con + 1
                
        print('Porcentaje de acierto', con/len(test)*100)

Nearest Neighbors
0.733540372671
neutral


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

No se está calculando el Gaussian Process?

In [21]:
print(db_7['Emotions'])

228    disgusted
219          sad
196        happy
132          sad
225    surprised
135      fearful
195    disgusted
Name: Emotions, dtype: object
